In [1]:
import sys
import os
# Add the src directory to Python path so model.py can find ssn and net modules
sys.path.append(os.path.abspath('../src'))

In [2]:
import numpy as np
from loguru import logger
# load the data
path = '../data_result/raw_data/gauss_cos_31x31.npy'# Initialize the weights
data = np.load(path)
logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

2025-09-22 23:54:05.747 | INFO     | __main__:<module>:6 - Loaded data with shape: (961,), dtype: [('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')]


In [3]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 1.0)
pruning_threshold = 1e-15

gamma = 5.0
alpha = 1e-5
lr_adam = 1e-5
regularization = (gamma, alpha) 
th = 0.0


In [4]:
from src.model import model
import torch
# Initialize the models
model_1 = model(
    activation=torch.relu, 
    power=power, 
    regularization=regularization, 
    optimizer='Adam', 
    loss_weights=loss_weights, 
    th = th
    )
model_2 = model(
    activation=torch.relu, 
    power=power, 
    regularization=regularization, 
    optimizer='SSN', 
    loss_weights=loss_weights, 
    th=th, 
    train_outerweights=True)

2025-09-22 23:54:09 | INFO     | src.model:_configure_logger:105 - Model initialized
2025-09-22 23:54:09 | INFO     | src.model:_configure_logger:105 - Model initialized


In [5]:
# prepare the data
data_train, data_valid = model_1._prepare_data(data)

2025-09-22 23:54:09 | INFO     | src.model:_prepare_data:150 - Training set: 864 samples, Validation set: 97 samples
2025-09-22 23:54:09 | INFO     | src.model:_prepare_data:153 - Data ranges - x: [-1.00, 1.00], v: [-0.72, 1.00], dv: [-5.86, 5.86]


In [6]:
from src.PDAP import retrain

best_iteration, history = retrain(
    data_train, 
    data_valid, 
    model_1, 
    model_2, 
    num_iterations, 
    M, 
    pruning_threshold
    )

2025-09-22 23:54:09 | INFO     | src.PDAP:retrain:56 - Iteration 0 - Starting...
2025-09-22 23:54:09 | INFO     | src.model:train:307 - Starting network training session
2025-09-22 23:54:09 | INFO     | src.model:_create_network:194 - Creating network with 50 neurons
2025-09-22 23:54:12 | INFO     | src.model:_setup_optimizer:244 - Using Adam optimizer with lr=0.01
2025-09-22 23:54:12 | INFO     | src.model:train:322 - Training hyperparameters: iterations=1000, batch_size=1620, display_every=100
2025-09-22 23:54:12 | INFO     | src.model:train:323 - Loss weights: value=1.0, gradient=1.0
2025-09-22 23:54:12 | INFO     | src.model:train:367 - Epoch 0: Train Loss = 9.419086, Val Loss = 10.576572
2025-09-22 23:54:13 | INFO     | src.model:train:367 - Epoch 100: Train Loss = 5.320490, Val Loss = 14.997412
2025-09-22 23:54:14 | INFO     | src.model:train:367 - Epoch 200: Train Loss = 3.950627, Val Loss = 21.297010
2025-09-22 23:54:15 | INFO     | src.model:train:367 - Epoch 300: Train Loss =

KeyboardInterrupt: 